In [ ]:
# Show the figures / plots inside the notebook
import numpy as np
import cv2
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from mpl_toolkits.mplot3d import Axes3D
import scipy.misc
from skimage.color import rgb2gray,rgb2hsv
from Utility_Functions import *
from skimage.exposure import histogram
from poisson_blending import *


face_cascade = cv2.CascadeClassifier('./haar/haarcascade_frontalface_default.xml')
mouth_cascade = cv2.CascadeClassifier('./haar/haarcascade_smile.xml')


# if mouth_cascade.empty():
#   raise IOError('Unable to load the mouth cascade classifier xml file')



Utility functions

In [ ]:
def show_hist(image, title, ignore_first=1, original_image=None):
    """
    This is a helper function to plot a histogram of an image using matplotlib
    """
    # Calculate the histogram
    hist = cv2.calcHist([image], [0], None, [256], [0, 256])
    hist[0:ignore_first] = 0
    if original_image is not None:
        peaks_indices, hist = get_peaks(hist, original_image)
    else:
        peaks_indices, hist = get_peaks(hist, image)
    print(peaks_indices)
    # Ignore the first 'ignore_first' values in the histogram
    

    # Plot the modified histogram
    plt.plot(hist, color='black')
    plt.xlabel('Pixel Value')
    plt.ylabel('Frequency')
    plt.title(title)
    plt.show()
    
def calcHist(img):

    hist = np.zeros((256,1))
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            hist[img[i,j]] += 1
    return hist
    
def showHist1(img):
    # An "interface" to matplotlib.axes.Axes.hist() method
    plt.figure()
    imgHist = histogram(img, nbins=256)
    
    bar(imgHist[1].astype(np.uint8), imgHist[0], width=0.8, align='center')
def show_image_3d(image):
    """
    This is a helper function to plot a 3D plot of the image
    """
    # Create the x and y coordinate arrays (here we just use pixel indices)
    xx, yy = np.mgrid[0:image.shape[0], 0:image.shape[1]]

    # Flatten xx and yy
    x_flat = xx.ravel()
    y_flat = yy.ravel()

    # Get the values for each pixel
    intensity = image.ravel()

    # Setup a figure and axes
    fig = plt.figure(figsize=(12, 10))
    ax = fig.add_subplot(111, projection='3d')

    # Plot the values
    ax.scatter(x_flat, y_flat, intensity, c=intensity, cmap='viridis')

    plt.show()
    

def remove_background(image, rectangle, iterations=5):
    """
    This is a helper function to use grabCut to remove the background from an image
    """
    #This line creates an empty binary mask with the same dimensions as the input image 
    #The mask is initially filled with zeros, indicating that no pixels are currently classified as either foreground or background.
    mask = np.zeros(image.shape[:2], np.uint8)
    cv2.grabCut(image, mask, rectangle, None, None, iterations, cv2.GC_INIT_WITH_RECT)

    #if mask is 2 or 0, it is converted to 0, otherwise converted to 1
    #2 or 0 means background, 1 or 3 means foreground
    mask = np.where((mask == 2) | (mask == 0), 0, 1).astype('uint8')

    #converting the input image to grayscale and applying the mask
    img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    img = img * mask

    return img

def divide_image(image):
    """
    This is a helper function to divide an image into 4 parts
    """
    img = image.copy()

    #this function takes an image and divide it into 4 parts
    part1 = img[0:img.shape[0]//2, 0:img.shape[1]//2]
    part2 = img[0:img.shape[0]//2, img.shape[1]//2:img.shape[1]]
    part3 = img[img.shape[0]//2:img.shape[0], 0:img.shape[1]//2]
    part4 = image[img.shape[0]//2:img.shape[0], img.shape[1]//2:img.shape[1]]

    return part1, part2, part3, part4

#this function takes an images and divide into 4 parts and calculate the histogram of each part
def local_hist(image):
    """
    This is a helper function that plots the histogram of each part of the image
    """
    part1, part2, part3, part4 = divide_image(image)

    show_hist(part1, 'Histogram of part1', original_image=image)
    show_hist(part2, 'Histogram of part2', original_image=image)
    show_hist(part3, 'Histogram of part3', original_image=image)
    show_hist(part4, 'Histogram of part4', original_image=image)

#this function takes array of threshold values and an image and return the image after applying the threshold to each part
def local_thresholding(thresholds, image):
    """
    This is a helper function that takes an array of threshold values 
    and an image and return the image after applying the threshold to each part
    """
    # Create a copy of the image
    image_thresholded = np.copy(image)

    # Divide the image into 4 parts
    part1 = image_thresholded[0:image.shape[0]//2, 0:image.shape[1]//2]
    part2 = image_thresholded[0:image.shape[0]//2, image.shape[1]//2:image.shape[1]]
    part3 = image_thresholded[image.shape[0]//2:image.shape[0], 0:image.shape[1]//2]
    part4 = image_thresholded[image.shape[0]//2:image.shape[0], image.shape[1]//2:image.shape[1]]

    # Apply thresholding
    image_thresholded[0:image.shape[0]//2, 0:image.shape[1]//2] = np.where(part1 < thresholds[0], 0, 255)
    image_thresholded[0:image.shape[0]//2, image.shape[1]//2:image.shape[1]] = np.where(part2 < thresholds[1], 0, 255)
    image_thresholded[image.shape[0]//2:image.shape[0], 0:image.shape[1]//2] = np.where(part3 < thresholds[2], 0, 255)
    image_thresholded[image.shape[0]//2:image.shape[0], image.shape[1]//2:image.shape[1]] = np.where(part4 < thresholds[3], 0, 255)
    

    return image_thresholded

def get_peaks(hist, image):
    """
    This is a helper function that takes and image and its histogram and return the peaks of the histogram
    """
    
    skin_color = image[0, 0]
    #tooth color is the average color of 5x5 pixels in the middle of the image
    tooth_color = image[(image.shape[0]//2)-40, image.shape[1]//2]
    print("dist here ",tooth_color, skin_color)
    dist = np.abs(np.abs(tooth_color.astype(np.float64)-skin_color.astype(np.float64))-10)
    dist = dist if dist >= 1 else 125

    #Here I am removing delta peaks
    #which are peaks that are much higher than the neighboring point
    hist_suppressed = suppress_delta_peaks(hist, percentage=0.45)

    peaks_indices, _ = find_peaks(hist_suppressed.flatten(), distance=dist, height=0.4*max(hist.flatten()))
    return peaks_indices, hist_suppressed


def get_peaks_local(image):
    """
    This is a helper function that takes an image and returns the peaks of the histogram of each part of the image
    """
    part1, part2, part3, part4 = divide_image(image)

    hist1 = cv2.calcHist([part1], [0], None, [256], [0, 256])
    hist2 = cv2.calcHist([part2], [0], None, [256], [0, 256])
    hist3 = cv2.calcHist([part3], [0], None, [256], [0, 256])
    hist4 = cv2.calcHist([part4], [0], None, [256], [0, 256])

    peak1, _ = get_peaks(hist1, image)
    peak2, _ = get_peaks(hist2, image)
    peak3, _ = get_peaks(hist3, image)
    peak4, _ = get_peaks(hist4, image)

    return peak1, peak2, peak3, peak4

def get_threshold_between_2_peaks(peak1, peak2, img):
    """
    This is a helper function that takes an image and 2 peaks and returns the threshold between them using Iterative Threshold Selection 
    """
    
    min_value = 255
    min_index = 0
    hist = cv2.calcHist([img], [0], None, [256], [0, 256])
    
    for i in range(0, peak2-peak1):
        if hist[i] < min_value:
            min_value = hist[i]
            min_index = i

    return min_index + peak1 + 40

def get_max_gap(hist):
    """
    This is a helper function that takes a histogram and returns the index and value of the maximum gap
    """
    max_gap = 0
    max_gap_index = 0
    for i in range(1, len(hist)-1):
        if hist[i] > hist[i-1] and hist[i] > hist[i+1]:
            if hist[i]-hist[i-1] > max_gap and hist[i]-hist[i+1] > max_gap:
                max_gap = hist[i]-hist[i-1]
                max_gap_index = i

    #for the first point
    if hist[0] > hist[1] and hist[0]-hist[1] > max_gap:
        max_gap = hist[0]-hist[1]
        max_gap_index = 0

    #for the last point
    if hist[-1] > hist[-2] and hist[-1]-hist[-2] > max_gap:
        max_gap = hist[-1]-hist[-2]
        max_gap_index = len(hist)-1

    return max_gap, max_gap_index

def suppress_delta_peaks(hist, percentage=0.5):
    """
    This is a helper function that takes a histogram and percentage and returns the histogran after suppressing the delta peaks
    """
    #Here I am removing delta peaks
    #the method is to remove peaks that are much higher than the neighboring peaks
    #I am calculating the difference between each peak and its left and right neighbors
    #if the difference is higher than a threshold, then I am removing this peak
    my_hist = hist.copy()   
    hist_max_gap = get_max_gap(hist)[0]
    threshold = int(hist_max_gap*percentage)

    for i in range(1, len(hist)-1):
        if hist[i] > hist[i-1] and hist[i] > hist[i+1]:
            if hist[i]-hist[i-1] > threshold and hist[i]-hist[i+1] > threshold:
                my_hist[i] = (hist[i-1] + hist[i+1])//2

    return my_hist

def getThreshold(img):
    # img = image.copy()
    # if (len(img.shape) == 3):
    #     img = rgb2gray(img)
    #     img = (img * 255).astype(np.uint8)
    hist = cv2.calcHist([img], [0], None, [256], [0, 256])
    imgHist = hist#calcHist(img)
    T_init = 0
    T = 0
    # Get the initial threshold Tinit (the mean of the image)
    for k in range(256):
        T_init += k*imgHist[k]
    T_init = np.round(T_init / np.sum(imgHist))
    
    # Get the two weighted averages (means), one for the lower pixels (have gray level less than Tinit) and the other for the higher pixels (have gray level more than Tinit)
    while True:
        sum1 = 0
        sum2 = 0
        count1 = 0
        count2 = 0
        for k in range(256):
            if k < T_init:
                sum1 += k*imgHist[k]
                count1 += imgHist[k]
            else:
                sum2 += k*imgHist[k]
                count2 += imgHist[k]
                
        m1 = sum1/ count1
        m2 = sum2/ count2
        
        T = np.round((m1 + m2) / 2)
        if T == T_init:
            break
        else:
            T_init = T
    return int(T)
  

Reading the image and applying face detection using haar cascades

In [ ]:
# Read the input image
image_path = './images/img1.jpg'
img = cv2.imread(image_path)

#resize image 716x900
img = cv2.resize(img, (716, 900))

def face_detection(img):

    # Perform face detection
    faces = face_cascade.detectMultiScale(img, scaleFactor=1.3, minNeighbors=5)

    return img, faces

def draw_rectangles(img, faces):
    # Draw rectangles around the detected faces
    for (x, y, w, h) in faces:
        cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)

    return img, faces

# # Display the result
# cv2.imshow('Face Detection', img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

Remove background using Grabcut algorithm


In [ ]:
img_temp = img.copy()
# img_temp = remove_background(img, faces[0], iterations=5)

# cv2.imshow("Foreground", img_temp)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

Segmenting the mouth area from the face and applying gamma correction

In [ ]:
#segment the face
def segment_face(img_temp, faces):
    img_temp = cv2.cvtColor(img_temp, cv2.COLOR_BGR2GRAY)
    for (x, y, w, h) in faces:
        #take the region of interest which is the face
        roi_gray = img_temp[y:y+h, x:x+w]
        
        #then take the mouth region only
        roi_gray_mouth_region = roi_gray[int(roi_gray.shape[0]/1.5):roi_gray.shape[0], int(roi_gray.shape[1]/4):int(roi_gray.shape[1]/4)+int(roi_gray.shape[1]/2)]

        # cv2.imshow('mouth Segmentation before gamma correction', roi_gray_mouth_region)

        #plot histogram
        show_hist(roi_gray_mouth_region, "before gamma correction")
        
        #apply gamma correction
        gamma = 1
        roi_gray_mouth_region = np.array(255 * (roi_gray_mouth_region / 255) ** gamma, dtype = 'uint8')

        #plot histogram
        show_hist(roi_gray_mouth_region, "after gamma correction")
    return roi_gray_mouth_region, roi_gray

# cv2.imshow('mouth Segmentation after gamma correction', roi_gray_mouth_region)

# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [ ]:
# local_hist(roi_gray_mouth_region)

Applying segmentation using threshholding

In [ ]:
#dividing the image into 4 parts and calculate the histogram of each part
def teeth_segmentation(roi_gray_mouth_region):
    part1, part2, part3, part4 = divide_image(roi_gray_mouth_region)
    peak1, peak2, peak3, peak4 = get_peaks_local(roi_gray_mouth_region)

    #puting the peaks in an array to make it easy to loop over them
    hist_peaks = [peak1, peak2, peak3, peak4]
    img_parts = [part1, part2, part3, part4]

    for i in range(len(hist_peaks)):
        #if there is one peak only, then put hist_peaks[i] = 255
        if len(hist_peaks[i]) == 1:
            hist_peaks[i] = 255
        else:
            hist_peaks[i] = get_threshold_between_2_peaks(hist_peaks[i][-2], hist_peaks[i][-1], img_parts[i][0])
            #hist_peaks[i] = getThreshold(img_parts[i][0]) 
            
    print(hist_peaks)
    roi_gray_mouth_region = local_thresholding(hist_peaks, roi_gray_mouth_region)
    return roi_gray_mouth_region


# cv2.imshow('After Threshholding', roi_gray_mouth_region)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [ ]:
# Color Segmentation
#notes
#isolate the skin color using hsv
#select teeth color using hsv

def color_segmentation(original_img_copy, roi_gray_mouth_region, faces):
    img_copy1 = cv2.cvtColor(original_img_copy, cv2.COLOR_BGR2RGB)


    for (x, y, w, h) in faces:
        # Take the region of interest which is the face
        face_region = img_copy1[y:y+h, x:x+w]
        
        # Then take the mouth region only
        mouth_region = face_region[int(face_region.shape[0]/1.5):face_region.shape[0],
                                    int(face_region.shape[1]/4):int(face_region.shape[1]/4)+int(face_region.shape[1]/2)]
        
    #convert to hsv color space to inhance the saturation
    mouth_region = cv2.cvtColor(mouth_region, cv2.COLOR_RGB2HSV)
    mouth_region[:,:,1] = mouth_region[:,:,1] * 1.3
    #decrease the value to make the image darker
    mouth_region[:,:,2] = mouth_region[:,:,2] * 0.7

    mouth_region = cv2.cvtColor(mouth_region, cv2.COLOR_HSV2RGB)

    show_images([mouth_region], ['Mouth Segmentation higher saturation'])

    Z = mouth_region.reshape((-1,3))
    # convert to np.float32
    Z = np.float32(Z)
    # define criteria, number of clusters(K) and apply kmeans()
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    K = 4
    ret,label,center=cv2.kmeans(Z,K,None,criteria,10,cv2.KMEANS_RANDOM_CENTERS)
    # Now convert back into uint8, and make original image
    center = np.uint8(center)
    res = center[label.flatten()]
    res2 = res.reshape((mouth_region.shape))

    show_images([res2], ['mouth segmentation Kmeans Output'])


    #plot histogram
    gray_img2 = cv2.cvtColor(res2, cv2.COLOR_BGR2GRAY)

    show_images([gray_img2], ['mouth segmentation Kmeans Output gray'])

    #apply gaussian filter
    gray_img2 = cv2.GaussianBlur(gray_img2, (5, 5), 5)


    #apply median filter
    #gray_img2 = cv2.medianBlur(gray_img2, 5)

    show_images([gray_img2], ['mouth segmentation'])
    imgHist = calcHist(gray_img2)

    print(imgHist.shape)

    brightest = 0
    for i in range(0, len(imgHist)):
        if imgHist[i][0] > 0:
            if i > brightest:
                brightest = i

    threshold = brightest
    print("threshold", threshold)

    new_thresholded_img = gray_img2.copy()

    new_thresholded_img = gray_img2 >= threshold


    #opening to remove noise

    # Define the dimensions of the ellipse (width and height)
    ellipse_width = 10
    ellipse_height = 5

    # Create an elliptical kernel
    ellipse_kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (ellipse_width, ellipse_height))

    new_thresholded_img = cv2.morphologyEx(new_thresholded_img.astype(np.uint8), cv2.MORPH_OPEN, ellipse_kernel)

    ellipse_width = 20
    ellipse_height = 10

    # Create an elliptical kernel
    ellipse_kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (ellipse_width, ellipse_height))


    #closing to fill holes
    new_thresholded_img = cv2.morphologyEx(new_thresholded_img.astype(np.uint8), cv2.MORPH_CLOSE, ellipse_kernel)

    #apply dilation to make the teeth thicker

    ellipse_width = 20
    ellipse_height = 10

    # Create an elliptical kernel
    ellipse_kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (ellipse_width, ellipse_height))

    new_thresholded_img = cv2.dilate(new_thresholded_img.astype(np.uint8), ellipse_kernel, iterations=6)



    #remove outliers from the image


    show_images([new_thresholded_img], ['teeth segmentation'])

    return new_thresholded_img, roi_gray_mouth_region, mouth_region



Applying opening to remove noise

In [ ]:
def apply_opening(roi_gray_mouth_region):
    kernel = np.ones((7, 7), np.uint8)

    # Define the dimensions of the ellipse (width and height)
    ellipse_width = 10
    ellipse_height = 5

    # Create an elliptical kernel
    ellipse_kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (ellipse_width, ellipse_height))

    # Apply opening operation to the binary mask
    opened_mask = cv2.morphologyEx(roi_gray_mouth_region, cv2.MORPH_OPEN, ellipse_kernel, iterations=2)

    roi_gray_mouth_region = roi_gray_mouth_region * opened_mask
    return roi_gray_mouth_region, opened_mask

# cv2.imshow("Opened Mask", roi_gray_mouth_region*255)  # Multiply by 255 for better visualization
# cv2.waitKey(0)
# cv2.destroyAllWindows()

Applying dilation to take the whole mouth

In [ ]:
def apply_dilation(roi_gray_mouth_region):
    kernel = np.ones((5, 5), np.uint8)

    # Define the dimensions of the ellipse (width and height)
    ellipse_width = 10
    ellipse_height = 5

    # Create an elliptical kernel
    ellipse_kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (ellipse_width, ellipse_height))


    # Apply closing operation to the binary mask
    dilate_mask = cv2.morphologyEx(roi_gray_mouth_region, cv2.MORPH_DILATE, ellipse_kernel, iterations=6)

    roi_gray_mouth_region = cv2.bitwise_or(roi_gray_mouth_region, dilate_mask)
    return roi_gray_mouth_region, dilate_mask

# cv2.imshow("Opened Mask", roi_gray_mouth_region*255)  # Multiply by 255 for better visualization
# cv2.waitKey(0)
# cv2.destroyAllWindows()

Apply closing to close gaps

In [ ]:
def apply_closing(roi_gray_mouth_region):
    kernel = np.ones((5, 5), np.uint8)

    # # Define the dimensions of the ellipse (width and height)
    # ellipse_width = 5
    # ellipse_height = 10

    # # Create an elliptical kernel
    # ellipse_kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (ellipse_width, ellipse_height))


    # Apply closing operation to the binary mask
    close_mask = cv2.morphologyEx(roi_gray_mouth_region, cv2.MORPH_CLOSE, kernel, iterations=5)
    roi_gray_mouth_region = cv2.bitwise_or(roi_gray_mouth_region, close_mask)
    return roi_gray_mouth_region, close_mask

# cv2.imshow("closemask", roi_gray_mouth_region*255)  # Multiply by 255 for better visualization
# cv2.waitKey(0)
# cv2.destroyAllWindows()

Plotting in 3d

In [ ]:
# show_image_3d(roi_gray_mouth_region)

Get the mean position of 1's


In [ ]:
def apply_mouth_translation(roi_gray_mouth_region):
    # Find the coordinates of all ones
    rows, cols = np.where(roi_gray_mouth_region == 1)

    img_center_y = roi_gray_mouth_region.shape[0]//2
    img_center_x = roi_gray_mouth_region.shape[1]//2

    # Calculate the mean position
    mean_row = np.mean(rows)
    mean_col = np.mean(cols)

    print(mean_row, mean_col)

    #I want to shift the image so that the mean position is in the center of the image
    shift_x = img_center_x - mean_col
    shift_y = img_center_y - mean_row

    # Define the translation matrix
    translation_matrix = np.float32([[1, 0, shift_x], [0, 1, shift_y]])

    # Translate the mask for anaylsis purposes
    img_translation = cv2.warpAffine(roi_gray_mouth_region, translation_matrix, (roi_gray_mouth_region.shape[1], roi_gray_mouth_region.shape[0]))

    show_image_3d(img_translation)

    return img_translation, translation_matrix
"""
NOTE: the tansalation will not happen on the mask, it has to applied after the mask is applied to the image
Here I am just showing the translation on the mask to make it easy to understand and analyze
"""

Take the mouth from the original image

In [ ]:
#Now we have the mouth region only
#We want to extract the lips from the orignal image
def extract_mouth(img, faces, roi_gray, roi_gray_mouth_region):
    for (x, y, w, h) in faces:
        
        mouth_region = img[y+int(roi_gray.shape[0]/1.5):y+h, x+int(roi_gray.shape[1]/4):x+int(roi_gray.shape[1]/4)+int(roi_gray.shape[1]/2)] 

        if mouth_region.shape[0] != roi_gray_mouth_region.shape[0] or mouth_region.shape[1] != roi_gray_mouth_region.shape[1]:
            print("Error: the mouth region and the mask are not the same size")
            continue
        #apply the mask to the mouth region
        #here we convert the mask to 3d to apply it to the mouth region
        #np.stack((roi_gray_mouth_region,) * 3, axis=-1) converts the mask to 3d in the last axis which is the color axis
        roi_gray_mouth_region_3d = np.stack((roi_gray_mouth_region,) * 3, axis=-1)
        mouth_region = mouth_region * roi_gray_mouth_region_3d
    return mouth_region
   
# cv2.imshow('mouth Segmentation', mouth_region)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

Translate image

In [ ]:
def translate_image(mouth_region, roi_gray_mouth_region, translation_matrix):
    mouth_region = cv2.warpAffine(mouth_region, translation_matrix, (roi_gray_mouth_region.shape[1], roi_gray_mouth_region.shape[0]))
    roi_gray_mouth_region = cv2.warpAffine(roi_gray_mouth_region, translation_matrix, (roi_gray_mouth_region.shape[1], roi_gray_mouth_region.shape[0]))

    # cv2.imshow('mouth Segmentation', roi_gray_mouth_region*255)

    #get left most point in the mask and the right most point and the top most point and the bottom most point
    #these points will be used to crop the image

    print(roi_gray_mouth_region.shape)
    left_most_point = (roi_gray_mouth_region.shape[0], roi_gray_mouth_region.shape[1])
    right_most_point = (0, 0)
    top_most_point = (roi_gray_mouth_region.shape[0], roi_gray_mouth_region.shape[1])
    bottom_most_point = (0, 0)



    print(left_most_point)

    for i in range(roi_gray_mouth_region.shape[0]):
        for j in range(roi_gray_mouth_region.shape[1]):
            if roi_gray_mouth_region[i][j] == 1:
                if i < left_most_point[0]:
                    left_most_point = (i, j)
                if i > right_most_point[0]:
                    right_most_point = (i, j)
                if j < top_most_point[1]:
                    top_most_point = (i, j)
                if j > bottom_most_point[1]:
                    bottom_most_point = (i, j)



    print (left_most_point, right_most_point, top_most_point, bottom_most_point)

    #crop the image using the points we got
    mouth_region = mouth_region[left_most_point[0]:right_most_point[0], top_most_point[1]:bottom_most_point[1]]
    roi_gray_mouth_region = roi_gray_mouth_region[left_most_point[0]:right_most_point[0], top_most_point[1]:bottom_most_point[1]]

    return mouth_region, roi_gray_mouth_region



# cv2.imshow('transalation', mouth_region)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [ ]:
# cv2.imshow('transalation', mouth_region)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

Loading the second image

In [ ]:
image2_path = './images/amin2.png'
img2 = cv2.imread(image2_path)

#resize image 716x900
img2 = cv2.resize(img2, (716, 1000))

##  ##
# Perform face detection
faces2 = face_cascade.detectMultiScale(img2, scaleFactor=1.3, minNeighbors=5)



# cv2.imshow('Face Detection 2', img2)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

Apply the mask to the second image

In [ ]:
def generate_output_image(img2, faces2, mouth_region, roi_gray_mouth_region):
    for (x2, y2, w2, h2) in faces2:

        #take the region of interest which is the face
        roi2 = img2[y2:y2+h2, x2:x2+w2]
        
        #and here I am taking the mouth region only, which will help me to detect the mouth region in the second image using haar
        #haar cascade works better with small images, it doesn't work well with large images
        #so I am taking the mouth region only to detect the mouth region in the second image
        mouth2_region = roi2[int(roi2.shape[0]/1.5):roi2.shape[0], int(roi2.shape[1]/4):int(roi2.shape[1]/4)+int(roi2.shape[1]/2)]

        #using haar cascade to detect the mouth region
        mouth2 = mouth_cascade.detectMultiScale(mouth2_region, scaleFactor=1.1, minNeighbors=5, minSize=(100, 100))

        # for (mx2, my2, mw2, mh2) in mouth2:
        #     cv2.rectangle(mouth2_region, (mx2, my2), (mx2+mw2, my2+mh2), (0, 255, 0), 2)


        [mx2, my2, mw2, mh2] = mouth2[0]
        mouth2_region = mouth2_region[my2:my2+mh2, mx2:mx2+mw2]


        #resize the mouth region to the same size of the mouth region of the second image
        mouth_region_resized = cv2.resize(mouth_region, (mw2, mh2))
        roi_gray_mouth_region_resized = cv2.resize(roi_gray_mouth_region, (mw2, mh2))


        im_mask = np.full(mouth2_region.shape, 255, dtype = np.uint8)
    
        center = (int(mouth_region_resized.shape[1]/2), int(mouth_region_resized.shape[0]/2))

        im_clone = cv2.seamlessClone(mouth2_region, mouth_region_resized, im_mask, center, cv2.MIXED_CLONE)

        # cv2.imshow("clone", im_clone)
        # cv2.waitKey(0)

        #USING THE SEAMLESS CLONE FUNCTION TO BLEND THE TWO IMAGES
        # put the mouth region in the second image
        # for i in range(mouth2_region.shape[0]):
        #     for j in range(mouth2_region.shape[1]):
        #         if roi_gray_mouth_region_resized[i][j] == 1:
        #             #blend colors
        #             mouth2_region[i][j] = im_clone[i][j]
        
        # put the mouth region in the second image
        for i in range(mouth2_region.shape[0]):
            for j in range(mouth2_region.shape[1]):
                if roi_gray_mouth_region_resized[i][j] == 1:
                    #blend colors
                    mouth2_region[i][j] = mouth_region_resized[i][j]
        
    
    img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
    mouth2_region = cv2.cvtColor(mouth2_region, cv2.COLOR_BGR2RGB)

    # cv2.imshow('mouth2_region', mouth2_region)
    # cv2.imshow('Result', img2)
    show_images([mouth2_region, img2], ['mouth2_region', 'Result'])

    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    return img2, mouth2_region



# cv2.imshow('mouth2_region', mouth2_region)
# cv2.imshow('Result', img2)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [ ]:
# GUI
import tkinter as tk
from tkinter import filedialog
from tkinter.filedialog import askopenfile
from PIL import Image, ImageTk

my_w = tk.Tk()
my_w.geometry("1500x800")  # Size of the window 
my_w.title('Image Processing Project')
my_font1 = ('times', 18, 'bold')

# Initialize separate global variables for each image
img1 = None
img2 = None
img3 = None

# Label and Button styling
label_style = {'width': 30, 'font': my_font1}
button_style = {'width': 20, 'bg': '#063462', 'fg': 'white', 'font': my_font1, 'borderwidth': 2, 'relief': 'groove', 'overrelief': 'ridge'}

l1 = tk.Label(my_w, text='Add the first Photo', **label_style)  
l1.grid(row=1, column=1)
b1 = tk.Button(my_w, text='Upload Image', command=lambda: upload_file1(), **button_style)
b1.grid(row=2, column=1, pady=10)

l2 = tk.Label(my_w, text='Add the second Photo', **label_style)  
l2.grid(row=1, column=2)
b2 = tk.Button(my_w, text='Upload Image', command=lambda: upload_file2(), **button_style)
b2.grid(row=2, column=2, pady=10)

b3 = tk.Button(my_w, text='Generate Output Image', command=lambda: generate_output(), **button_style)
b3.grid(row=2, column=3, pady=10)

def upload_file1():
    global img1, filename1
    f_types = [('Image Files', '*.png;*.gif;*.ppm;*.pgm;*.pbm;*.tiff;*.bmp;*.jpeg;*.jpg')]
    filename1 = filedialog.askopenfilename(filetypes=f_types)
    if filename1:
        img1 = Image.open(filename1)
        img_resized = img1.resize((400, 600))  # new width & height
        img1 = ImageTk.PhotoImage(img_resized)
        b2 = tk.Button(my_w, image=img1)  # using Button
        b2.grid(row=3, column=1)

def upload_file2():
    global img2, filename2
    f_types = [('Image Files', '*.png;*.gif;*.ppm;*.pgm;*.pbm;*.tiff;*.bmp;*.jpeg;*.jpg')]
    filename2 = filedialog.askopenfilename(filetypes=f_types)
    if filename2:
        img2 = Image.open(filename2)
        img_resized = img2.resize((400, 600))  # new width & height
        img2 = ImageTk.PhotoImage(img_resized)
        b2 = tk.Button(my_w, image=img2)  # using Button
        b2.grid(row=3, column=2)

def generate_output():
    # if this button is clicked, then the output image will be generated
    global img1, img2, img3
    if img1 and img2:
        img1_copy = cv2.imread(filename1)
        img2_copy = cv2.imread(filename2)

        img1_copy = cv2.resize(img1_copy, (716, 900))
        img2_copy = cv2.resize(img2_copy, (716, 1000))

        img1_copy, faces1 = face_detection(img1_copy)
        img1_copy, faces1 = draw_rectangles(img1_copy, faces1)
        roi_gray_mouth_region1, roi_gray1 = segment_face(img1_copy, faces1)

        local_hist(roi_gray_mouth_region1)
        
        roi_gray_mouth_region1 = teeth_segmentation(roi_gray_mouth_region1)
        
        new_thresholded_img, roi_gray_mouth_region1, mouth_region1 = color_segmentation(img1_copy, roi_gray_mouth_region1, faces1)

        roi_gray_mouth_region1, opened_mask1 = apply_opening(roi_gray_mouth_region1)
        roi_gray_mouth_region1, dilate_mask1 = apply_dilation(roi_gray_mouth_region1)
        roi_gray_mouth_region1, close_mask1 = apply_closing(roi_gray_mouth_region1)

        roi_gray_mouth_region1 = new_thresholded_img

        roi_gray_mouth_region1, translation_matrix1 = apply_mouth_translation(roi_gray_mouth_region1)

        mouth_region1 = extract_mouth(img1_copy, faces1, roi_gray1, roi_gray_mouth_region1)

        mouth_region1, roi_gray_mouth_region1 = translate_image(mouth_region1, roi_gray_mouth_region1, translation_matrix1)

        img2_copy, faces2 = face_detection(img2_copy)

        img3, mouth2_region = generate_output_image(img2_copy, faces2, mouth_region1, roi_gray_mouth_region1)

        img3 = cv2.cvtColor(img3, cv2.COLOR_BGR2RGB)
        # cv2.imshow('Result', img3)
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()

        # Convert the OpenCV image to a PIL Image
        img3_pil = Image.fromarray(cv2.cvtColor(img3, cv2.COLOR_BGR2RGB))
        img3_pil = img3_pil.resize((400, 600))  # new width & height

        # Convert the PIL Image to PhotoImage
        img3 = ImageTk.PhotoImage(img3_pil)

        # Use Label widget to display the image
        label_img3 = tk.Label(my_w, image=img3)
        label_img3.grid(row=3, column=3)

        # img3 = ImageTk.PhotoImage(img3)
        # b3 = tk.Button(my_w, image=img3)  # using Button
        # b3.grid(row=3, column=3)

my_w.mainloop()  # Keep the window open